# PubMed Search + Fetch (Dementia Caregiving)

Set your email and optional NCBI API key as environment variables before running.

To create an API key, sign in at https://account.ncbi.nlm.nih.gov/settings/ and generate an access key.

Example env var setup:
- `export NCBI_EMAIL="you@example.com"` (bash/zsh)
- `export NCBI_API_KEY="YOUR_KEY"` (bash/zsh)
- `export S3_BUCKET="your-bucket"` (bash/zsh)

Optional: create a local `.env` (gitignored) and load it with `python-dotenv`.

Reference: https://biopython.org/docs/latest/Tutorial/chapter_entrez.html#esearch-searching-the-entrez-databases

In [ ]:
import os

from Bio import Entrez

try:
    from dotenv import load_dotenv

    load_dotenv()
except Exception:
    pass

EMAIL = os.getenv("NCBI_EMAIL")
API_KEY = os.getenv("NCBI_API_KEY", "")

if not EMAIL:
    raise ValueError("NCBI_EMAIL must be set in your environment or .env")

Entrez.email = EMAIL
if API_KEY:
    Entrez.api_key = API_KEY

# Using MeSH (Medical Subject Headings) for broader recall, plus tiab
# (Title/Abstract) terms to keep results focused and more current.
QUERY = (
    '("Dementia"[Mesh] OR "Mild Cognitive Impairment"[Mesh]) '
    'AND ("Decision Support Systems, Clinical"[Mesh] OR "Caregivers"[Mesh] '
    'OR caregiver*[tiab] OR "decision support"[tiab])'
)
RETMAX = 500

# Using usehistory to enable lazy fetches (WebEnv + QueryKey).
# Docs: https://biopython.org/docs/latest/Tutorial/chapter_entrez.html#esearch-searching-the-entrez-databases
try:
    stream = Entrez.esearch(db="pubmed", term=QUERY, retmax=RETMAX, usehistory="y")
    record = Entrez.read(stream)
    stream.close()
except Exception as exc:
    raise RuntimeError(f"PubMed search failed: {exc}")

summary = {
    "count": int(record.get("Count", 0)),
    "retmax": int(record.get("RetMax", 0)),
    "returned_ids": len(record.get("IdList", [])),
    "query_translation": record.get("QueryTranslation"),
    "webenv": record.get("WebEnv"),
    "query_key": record.get("QueryKey"),
}
summary


In [ ]:
# Lazy fetch: stream up to RETMAX records using the search history, in chunks.
import os
import time

from Bio import Medline

FETCH_DIR = "data/pubmed_fetch"
os.makedirs(FETCH_DIR, exist_ok=True)

webenv = record.get("WebEnv")
query_key = record.get("QueryKey")

total_count = int(record.get("Count", 0))
target_count = min(RETMAX, total_count)

batch_size = 100
request_delay = 0.10 if API_KEY else 0.34

def iter_fetch_batches():
    if not (webenv and query_key):
        return
    for start in range(0, target_count, batch_size):
        stream = Entrez.efetch(
            db="pubmed",
            rettype="medline",
            retmode="text",
            retstart=start,
            retmax=min(batch_size, target_count - start),
            webenv=webenv,
            query_key=query_key,
        )
        try:
            yield stream
        finally:
            stream.close()
        if start + batch_size < target_count:
            time.sleep(request_delay)

def format_record(rec):
    parts = []
    if rec.get("PMID"):
        parts.append(f"PMID: {rec['PMID']}")
    if rec.get("TI"):
        parts.append(f"Title: {rec['TI']}")
    if rec.get("AU"):
        parts.append(f"Authors: {', '.join(rec['AU'])}")
    if rec.get("JT"):
        parts.append(f"Journal: {rec['JT']}")
    if rec.get("DP"):
        parts.append(f"Date: {rec['DP']}")
    if rec.get("AB"):
        parts.append(f"Abstract:\n{rec['AB']}")
    return "\n".join(parts).strip()

written = 0
for stream in iter_fetch_batches() or []:
    for rec in Medline.parse(stream):
        pmid = rec.get("PMID")
        if not pmid:
            continue
        text = format_record(rec)
        if not text:
            continue
        out_path = os.path.join(FETCH_DIR, f"{pmid}.txt")
        with open(out_path, "w", encoding="utf-8") as handle:
            handle.write(text)
        written += 1

f"Wrote {written} records to {FETCH_DIR}."

In [ ]:
# Cleanup: delete fetched files.
import shutil

if os.path.isdir(FETCH_DIR):
    shutil.rmtree(FETCH_DIR)
    "Deleted fetched data folder."
else:
    "Nothing to delete."

In [ ]:
# Upload fetched files to the raw prefix in S3.
import os

import boto3

S3_BUCKET = os.getenv("S3_BUCKET")
RAW_PREFIX = "raw"

if not S3_BUCKET:
    raise ValueError("S3_BUCKET must be set in your environment or .env")

s3 = boto3.client("s3")

uploaded = 0
for filename in os.listdir(FETCH_DIR):
    if not filename.endswith(".txt"):
        continue
    local_path = os.path.join(FETCH_DIR, filename)
    key = f"{RAW_PREFIX}/{filename}"
    s3.upload_file(local_path, S3_BUCKET, key)
    uploaded += 1

f"Uploaded {uploaded} files to s3://{S3_BUCKET}/{RAW_PREFIX}/"